In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [9]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
y_pred

array([18.52778307, 23.06578208, 33.68635854, ..., 11.89045938,
       15.10268128,  9.46059157])

## Question 1

In [10]:
np.std(y_pred)

5.28140357655334

## Question 2

In [25]:
year = 2022
month = 2

In [26]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [27]:
df['ride_id'].to_numpy()

array(['2022/02_0', '2022/02_1', '2022/02_2', ..., '2022/02_2979428',
       '2022/02_2979429', '2022/02_2979430'], dtype=object)

In [28]:
df_result = pd.DataFrame(data={'ride_id': df['ride_id'].to_numpy(), 'prediction': y_pred})

In [29]:
df_result

,ride_id,prediction
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2918182,2022/02_2979426,12.038225
2918183,2022/02_2979427,11.441569
2918184,2022/02_2979428,11.890459
2918185,2022/02_2979429,15.102681


In [22]:
df_result.to_parquet(
    'results.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
# 58M